<a href="https://colab.research.google.com/github/dhanavadh/NLP_2024/blob/main/Sentiment_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis with Logistic Regression

In this activity, you will

- Explore the data to understand the dataset

- We want to know what keyword with positive sentiment (5 star reviews) and neutral sentiment (3 star reviews) and negative sentiment (1 star reviews)

- We also want to know where the model falls short. Analyze the errors that your model makes


** Steps

1) Download data
2) Open
	- How many rows?
	- How many columns? What are they?
3) Preprocess
	- Label 1 star --> negative 3 -->neutral  5--> positive
	- Discard 2 and 4 star reviews
4) Split train test (Think how many)
5) Make feature vector (vectorize -- CountVectorizer)
6) Train logistic regression on training set
7) Evaluate on test set
8) Predict on test set and add a column to test set
     | text | gold standard label | predicted label |
     |------|---------------------|-----------------|
9) Save to Excel
10) Open the prediction file in Excel and find error patterns

In [ ]:
!gdown --id 1UpSJkqHhU7RMQC47Ii-6VZuByPYfhLrj